# Day 1a: From Prompt to Action（LangChain可本地运行版）

🚀 **你的第一个 AI Agent: 从 Prompt 到 Action**

欢迎来到 5 天 Agent 课程的第一天!

本 Notebook 是你构建 AI Agent 的第一步。Agent 不仅仅能响应 prompt —— 它还可以采取**行动**来查找信息或完成任务。

在本 Notebook 中，你将:

- ✅ 学习如何配置 API Key 并初始化 LLM
- ✅ 构建你的第一个简单 Agent
- ✅ 运行 Agent 并观察它如何使用工具（如网络搜索）来回答问题

---

**原版**: Google ADK Team (Kaggle 5-Day Agent Course)  
**改写**: LangChain 实现
> **ℹ️ 声明**: 本教程改编自 Google [5-Day AI Agents Course](https://www.kaggle.com/learn-guide/5-day-agents)，由 LangChain 重写以适配本地运行。内容仅供学习研究。

## ⚙️ Section 1: Setup (环境设置)

### 1.1 切换内核
- 点击右上角的 Kernel (内核) 选择区域。
- 在列表中选择 Python (5-DAY-AGENTS)。

> 关于如何初始化环境和创建内核，请查看 [README 如何运行](../README.md#-如何运行)

### 1.2 安装依赖

In [ ]:
# 优选 uv 包管理器 
!uv pip install langchain langchain-openai langchain-community python-dotenv duckduckgo-search tavily-python

### 1.3 导入依赖库

In [ ]:
import os
import json
import warnings
from dotenv import load_dotenv, find_dotenv

# LangChain 核心组件
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent

# 搜索工具
from langchain_community.tools import DuckDuckGoSearchResults, TavilySearchResults

# 忽略废弃警告
from langchain_core._api.deprecation import LangChainDeprecationWarning
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)
warnings.filterwarnings("ignore")

print("✅ 依赖库导入成功")

### 1.4 配置 API Key

> 关于 .env 文件内容说明，请查看 [README 环境配置文件 (.env) 说明](../README.md#3-环境配置文件-env-说明)

In [ ]:
# 加载环境变量
load_dotenv(find_dotenv())

def configure_api_keys():
    """加载并验证 API 配置"""
    model_name = os.environ.get("MODEL_NAME")
    api_key = os.environ.get("OPENAI_API_KEY")
    base_url = os.environ.get("OPENAI_API_BASE")
    tavily_key = os.environ.get("TAVILY_API_KEY")

    if not api_key:
        print("❌ 错误: 未在 .env 文件中找到 OPENAI_API_KEY")
        return None
    
    # 打印部分 Key 以验证 (隐藏敏感信息)
    masked_key = f"{api_key[:8]}..." if api_key else "None"
    print(f"✅ API Key 已加载: {masked_key}")
    print(f"✅ 使用模型: {model_name}")
    if base_url:
        print(f"✅ API Base URL: {base_url}")
    
    if tavily_key:
        print("✅ Tavily Search API 已启用 (增强搜索能力)")
    else:
        print("⚠️ 未找到 Tavily Key，将使用 DuckDuckGo 搜索 (免费但质量可能受限)")
    
    return {
        "model_name": model_name,
        "api_key": api_key,
        "base_url": base_url,
        "tavily_key": tavily_key,
        "extra_body": json.loads(os.environ.get("EXTRA_BODY", "{}"))
    }

config = configure_api_keys()

### 1.5 初始化 LLM

现在，让我们初始化 LangChain 的 ChatOpenAI 模型。这相当于 ADK 中的 `Gemini` 模型配置。

In [ ]:
# 初始化 LLM
llm = ChatOpenAI(
    model=config["model_name"],
    api_key=config["api_key"],
    base_url=config["base_url"],
    temperature=0.7,
    model_kwargs={"extra_body": config["extra_body"]}
)

print(f"✅ LLM 初始化完成: {config['model_name']}")

## 🤖 Section 2: Your First AI Agent (你的第一个 AI Agent)

### 🤔 2.1 什么是 AI Agent?

你可能之前使用过 LLM（如 GPT、Gemini），你给它一个 prompt，它返回一个文本响应：

```
Prompt -> LLM -> Text
```

AI Agent 更进一步。Agent 可以**思考**、**采取行动**，并**观察**这些行动的结果，然后给你一个更好的答案：

```
Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer
```

在本 Notebook 中，我们将构建一个可以执行**网络搜索**的 Agent。让我们看看它和普通 LLM 的区别！

### 🛠️ 2.2 定义工具 (Tools)

Agent 需要**工具 (Tools)** 来与外部世界交互。在 ADK 中使用的是 `google_search`，而我们这里使用 LangChain 支持的搜索工具。

我们定义一个 `internet_search` 工具，它会：
1. 优先使用 Tavily Search（如果配置了 API Key）
2. 否则降级使用免费的 DuckDuckGo Search

In [ ]:
@tool
def internet_search(query: str) -> str:
    """
    Performs an internet search using Tavily (preferred) or DuckDuckGo (fallback).
    Use this tool to find answer about current events, weather, or specific facts.
    
    Args:
        query: The search query to look up on the internet.
    """
    # 优先尝试 Tavily (如果可用)
    if config["tavily_key"]:
        try:
            print(f"    🔍 [Tool Call] 正在调用 Tavily 搜索: '{query}'...")
            search = TavilySearchResults(k=5)
            results = search.run(query)
            print(f"    ✅ [Tool Return] 搜索成功，获取了相关结果。")
            return results
        except Exception as e:
            print(f"    ⚠️ [Warning] Tavily 搜索失败: {e}")
    
    # 降级到 DuckDuckGo
    try:
        print(f"    🔍 [Tool Call] 正在调用 DuckDuckGo 搜索: '{query}'...")
        search = DuckDuckGoSearchResults(backend="api")
        results = search.run(query)
        print(f"    ✅ [Tool Return] 搜索成功。")
        return results
    except Exception as e:
        return f"搜索失败: {e}"

# 定义工具列表
tools = [internet_search]

print("✅ 搜索工具 `internet_search` 已就绪。")

### 2.3 定义 Agent

现在，让我们构建 Agent。我们需要配置以下属性：

| ADK 概念 | LangChain 对应 |
|---------|---------------|
| `name` | 用于日志/调试 |
| `model` | `ChatOpenAI` (我们的 LLM) |
| `instruction` | System Prompt (在 `ChatPromptTemplate` 中) |
| `tools` | 工具列表 (我们的 `internet_search`) |

在 LangChain 中，Agent 由三部分组成：
1. **LLM** (大脑 - The Brain)
2. **Tools** (身体 - The Body)
3. **Prompt** (灵魂 - The Soul)

In [ ]:
# 定义 Prompt 模板
# 这相当于 ADK 中的 instruction: "You are a helpful assistant. Use Google Search for current info or if unsure."
prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一个乐于助人的 AI 助手。请始终用中文回答用户的问题。
    
对于以下类型的问题，请务必使用搜索工具获取最新信息：
- 当前事件和新闻
- 天气查询
- 你不确定的事实性问题

如果搜索结果是英文的，请翻译并综合成中文回答。"""),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# 创建 Agent
agent = create_tool_calling_agent(llm, tools, prompt)

print("✅ Agent 已定义 (Brain + Body + Soul)")

### 2.4 创建 Runner (运行器)

在 ADK 中，使用 `InMemoryRunner` 来管理对话和执行 Agent。

在 LangChain 中，对应的是 `AgentExecutor`：

In [ ]:
# 创建执行器 (AgentExecutor)
# handle_parsing_errors=True 可以防止模型输出格式不正确时导致崩溃
runner = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=False,  # 设为 True 可以看到详细推理过程
    handle_parsing_errors=True
)

print("✅ Runner 已创建。")

### 2.5 运行 Agent

现在让我们运行 Agent！我们会问两个问题：

1. **知识性问题**: 关于 Agent Development Kit 的介绍（Agent 可能需要搜索来获取最新信息）
2. **实时信息**: 当前天气（Agent 必须使用搜索工具）

---

**演示 1: 知识检索**

In [ ]:
# 第一个问题 - 关于 ADK
query1 = "What is Agent Development Kit from Google? What languages is the SDK available in? "
print(f"User > {query1}\n")
print("Agent 正在思考...\n")

try:
    response1 = runner.invoke({"input": query1 + " (请务必用中文回答)"})   
    print(f"\n🤖 Agent 回答:\n{response1['output']}")
except Exception as e:
    print(f"❌ 运行出错: {e}")

### 2.6 Agent 是如何工作的？

Agent 执行了搜索来获取关于 ADK 的最新信息，它之所以知道使用工具，是因为：

1. **工具感知**: Agent 会检查并意识到它有哪些可用工具
2. **指令遵循**: Agent 的 System Prompt 指示它在不确定或需要当前信息时使用搜索工具

这就是 **ReAct 模式** (Reasoning + Acting) 的核心思想 - Agent 不只是生成文本，而是能够思考、行动并观察结果。

---

**演示 2: 实时数据查询 (天气)**

In [ ]:
# 第二个问题 - 天气（需要实时数据）
query2 = "What's the weather in London?"
print(f"User > {query2}\n")
print("Agent 正在思考 (这需要调用搜索工具)...\n")

try:
    response2 = runner.invoke({"input": query2 + " (请务必用中文回答)"})
    print(f"\n🤖 Agent 回答:\n{response2['output']}")
except Exception as e:
    print(f"❌ 运行出错: {e}")

## 🚀 Section 3: Your Turn! (轮到你了)

现在是你见证 Agent 实际行动的机会了！问它一个需要当前信息的问题。

试试以下问题，或者自己编一个：

- Who won the last soccer world cup?
- What new movies are showing in theaters now?
- 今天有什么重要新闻？

In [ ]:
# 你的问题
your_query = "Who won the last soccer world cup?"  # 修改这里问你想问的问题

print(f"User > {your_query}\n")
print("Agent 正在思考...\n")

try:
    response = runner.invoke({"input": your_query+ " (请务必用中文回答)"})
    print(f"\n🤖 Agent 回答:\n{response['output']}")
except Exception as e:
    print(f"❌ 运行出错: {e}")

## ✅ 恭喜! (Congratulations!)

你已经构建并运行了你的第一个 LangChain Agent！你刚刚见证了 Agent 开发的核心概念。

**关键收获**:

> 你的 Agent 不仅仅是响应 —— 它**推理**出需要更多信息，然后**行动**使用工具获取信息。这种**采取行动的能力**是所有基于 Agent 的 AI 的基础。

---

### 🔜 下一步

在 Day 1b 中，我们将探索更多的 Agent 模式，包括：
- 多 Agent 协作
- 顺序执行 vs 并行执行
- 循环 Agent 模式

继续加油！🎉